In [1]:
import pandas as pd
import os
%pwd

u'/Users/Driz/capstone'

In [2]:
df = pd.read_csv('gender_included_df.csv', encoding='utf-8')

/Users/Driz/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (4,5,6,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count,gender
0,9549,32,209,"Miss Hoover: No, actually, it was a little of ...",848000,True,464,3.0,Miss Hoover,Springfield Elementary School,"No, actually, it was a little of both. Sometim...",no actually it was a little of both sometimes ...,31.0,f
1,9550,32,210,Lisa Simpson: (NEAR TEARS) Where's Mr. Bergstrom?,856000,True,9,3.0,Lisa Simpson,Springfield Elementary School,Where's Mr. Bergstrom?,wheres mr bergstrom,3.0,f
2,9551,32,211,Miss Hoover: I don't know. Although I'd sure l...,856000,True,464,3.0,Miss Hoover,Springfield Elementary School,I don't know. Although I'd sure like to talk t...,i dont know although id sure like to talk to h...,22.0,f
3,9552,32,212,Lisa Simpson: That life is worth living.,864000,True,9,3.0,Lisa Simpson,Springfield Elementary School,That life is worth living.,that life is worth living,5.0,f
4,9553,32,213,Edna Krabappel-Flanders: The polls will be ope...,864000,True,40,3.0,Edna Krabappel-Flanders,Springfield Elementary School,The polls will be open from now until the end ...,the polls will be open from now until the end ...,33.0,f


In [4]:
# create new df reducing to column for further preprocessing and my y-column

df_reduced = df[['normalized_text','gender']]
                
df_reduced.head()

,normalized_text,gender
0,no actually it was a little of both sometimes ...,f
1,wheres mr bergstrom,f
2,i dont know although id sure like to talk to h...,f
3,that life is worth living,f
4,the polls will be open from now until the end ...,f


In [8]:
df_reduced.dropna(inplace=True)

/Users/Driz/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [9]:
# need to remove stop words
import nltk
from nltk.corpus import stopwords

stop_words = nltk.corpus.stopwords.words('english')
# stop_words = stopwords.words('english')
filtered_sentence = [w for w in df_reduced['normalized_text'].values if not w in stop_words]
#print filtered_sentence 

In [10]:
stop_words

[u'i',
 u'me',
 u'my',
 u'myself',
 u'we',
 u'our',
 u'ours',
 u'ourselves',
 u'you',
 u'your',
 u'yours',
 u'yourself',
 u'yourselves',
 u'he',
 u'him',
 u'his',
 u'himself',
 u'she',
 u'her',
 u'hers',
 u'herself',
 u'it',
 u'its',
 u'itself',
 u'they',
 u'them',
 u'their',
 u'theirs',
 u'themselves',
 u'what',
 u'which',
 u'who',
 u'whom',
 u'this',
 u'that',
 u'these',
 u'those',
 u'am',
 u'is',
 u'are',
 u'was',
 u'were',
 u'be',
 u'been',
 u'being',
 u'have',
 u'has',
 u'had',
 u'having',
 u'do',
 u'does',
 u'did',
 u'doing',
 u'a',
 u'an',
 u'the',
 u'and',
 u'but',
 u'if',
 u'or',
 u'because',
 u'as',
 u'until',
 u'while',
 u'of',
 u'at',
 u'by',
 u'for',
 u'with',
 u'about',
 u'against',
 u'between',
 u'into',
 u'through',
 u'during',
 u'before',
 u'after',
 u'above',
 u'below',
 u'to',
 u'from',
 u'up',
 u'down',
 u'in',
 u'out',
 u'on',
 u'off',
 u'over',
 u'under',
 u'again',
 u'further',
 u'then',
 u'once',
 u'here',
 u'there',
 u'when',
 u'where',
 u'why',
 u'how',
 u'all

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english',
                       ngram_range=(1,3))
tfidf_matrix = tfidf.fit_transform(df_reduced['normalized_text'].values)

In [27]:
from sklearn.preprocessing import LabelEncoder

y = LabelEncoder().fit_transform(df_reduced['gender'].values)

In [46]:
import numpy as np
np.mean(y)

0.74062486514478054

In [28]:
from sklearn.cross_validation import train_test_split

x_train, x_test, y_train, y_test = train_test_split(tfidf_matrix, y)

In [ ]:
# do label encoder before fed to train test

In [15]:
# these shapes pertain the filtered_sentence. there is discrepancy in the shape causing it to not work

df_reduced.shape

(92692, 2)

In [16]:
tfidf_matrix.shape

(92112, 576162)

In [18]:
len(filtered_sentence)

92112

In [35]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1)
rf.fit(x_train, y_train)
predictions = rf.predict(x_test)
rf.score(x_test, y_test)

0.74517757735295387

In [36]:
# rf.score(predictions, y_test)
predictions.shape

(23173,)

In [37]:
y_test.shape

(23173,)

In [38]:
predictions

array([1, 0, 1, ..., 1, 1, 1])

In [39]:
y_test

array([1, 0, 1, ..., 1, 1, 0])

In [52]:
from sklearn.metrics import confusion_matrix
#import matplotlib.pyplot as plt --to make the fancy style confusion matrix

confusion = pd.DataFrame(confusion_matrix(y_test, predictions), columns=['predicted_f', 'predicted_m'], index=['is_f', 'is_m'])
confusion

,predicted_f,predicted_m
is_f,963,4957
is_m,948,16305


In [ ]:
Looks like my out-of-box model overly predicts male characters...